**Overview**

APScheduler is a task scheduling and management system written in Python.

Thinking of it as a cron/at daemon running inside your application is not far off, but APScheduler also provides management and monitoring of jobs, and much more.



```
<agronholm> well, because I had a Java project that relied on Quartz and the only scheduling alternative on Python was Celery
<agronholm> I was porting my code over to Python
<agronholm> and Celery was way overkill
...
<agronholm> my intention is to create a new job queueing/scheduling component for my Asphalt framework
<agronholm> when done I expect it to be superior to celery in every way
    ...
<agronholm> asphalt is an asyncio based framework where you can mix async and blocking code
```

**Interesting points**

    * Event listener

**The author**

Alex Grönholm, the main author of APScheduler is self employed as a contractor. 

He is living in Finland.

He developed APScheduler while he working on a Java project which relied on Quartz. 

He wanted to migrate everything to Python, but the only scheduling alternative on Python was Celery

**What about Celery**

Celery is a distributed task queue with basic scheduling capabilities, while APScheduler is a full featured scheduler with basic task queuing capabilities. 

APScheduler tend to be easier to setup and to integrate with an existing solution.

**How to Install**

$ pip install apscheduler

**How does it work**

You define a series of jobs and the rules about when they should run

**Interval**

In [4]:
from apscheduler.schedulers.background import BackgroundScheduler

def myfunc():
    pass

def myfunc2():
    pass

scheduler = BackgroundScheduler()
scheduler.add_job(myfunc, 'interval', minutes=2, id='my_job_id')

**Cron-like**

In [ ]:
scheduler.add_job(myfunc, trigger='cron', minute='*', second="*/5")
scheduler.add_job(myfunc2, trigger='cron', day_of_week="2")
scheduler.add_job(myfunc2, trigger='cron', day_of_week="sat,sun")

# More informations at http://apscheduler.readthedocs.org/en/latest/modules/triggers/cron.html?highlight=cron

**Date-based**

In [ ]:
sched.add_job(myfunc, 'date', run_date=date(2009, 11, 6), args=['text'])

**Thread and Process Pool**

APScheduler could also be used to run long running task in the background

In [ ]:
from apscheduler.executors.pool import ProcessPoolExecutor
from apscheduler.schedulers.background import BackgroundScheduler

executors = {
    'default': {'type': 'threadpool', 'max_workers': 20},
    'processpool': ProcessPoolExecutor(max_workers=5)
}
job_defaults = {
    'coalesce': False,
    'max_instances': 3
}

def my_producer_job():
    while True:
        # produce stuff and save it the database
        
def my_consumer_job():
    # read from the database
        
                
scheduler = BackgroundScheduler()

producer = scheduler.add_job(my_producer_job, name="producer", executor="default")
consumer = scheduler.add_job(my_consumer_job, name="consumer", executor="processpool")

scheduler.configure(executors=executors, job_defaults=job_defaults, timezone=utc)

scheduler.start()

**Advanced functionalities**

In [ ]:
# event listener

def my_listener(event):
    if event.exception:
        print('The job crashed :(')
    else:
        print('The job worked :)')

scheduler.add_listener(my_listener, EVENT_JOB_EXECUTED | EVENT_JOB_ERROR)

In [8]:
# custom trigger

import random

from apscheduler.triggers.base import BaseTrigger

class MyTrigger(BaseTrigger):
    def get_next_fire_time(self, previous_fire_time, now):
        random_value = random.randint(1, 10)
        if (now - previous_fire_time).seconds > random_value:
            return datetime.datetime(now.replace(minute=random_value))
trigger = MyTrigger()
job = scheduler.add_job(myfunc, trigger, id="my_job_id")

<Job (id=f1896b045c2e4e198c3ad5fd34a9fb5e name=myfunc)>

In [ ]:
# Limiting the number of concurrently executing instances

job.modify(max_instances=6)

In [ ]:
# reschedule an existing job

scheduler.reschedule_job('my_job_id', trigger='cron', minute='*/5')

In [ ]:
# shuting down the scheduler

scheduler.shutdown()

# or more violently

scheduler.shutdown(wait=False)